# CMSC 320 - Final Tutorial - UMD Subreddit

In [2]:
import requests

In [5]:
def make_get_request(URL, given_params=None):
    """Sends a GET request to the given URL.
    
    Parameters
    ----------
    URL : str
        The url to send a GET request
    given_params : dictionary, optional
        A dictionary of any additional parameters (default is None)
        
        
    Returns
    -------
    dictionary
        A dictionary containing the JSON response
    """
    
    SUCCESS = 200
    response = requests.get(URL, params=given_params)
    
    if (response.status_code == SUCCESS):
        return response.json()
    else:
        return {};

In [6]:
professors_json = make_get_request("https://api.umd.io/v1/professors")
print(professors_json)

[{'name': 'A Anthony', 'taught': [{'course_id': 'AMST203', 'semester': '201905'}]}, {'name': 'Aaron Bartlett', 'taught': [{'course_id': 'ENGL101', 'semester': '201908'}]}, {'name': 'Aaron Doliana', 'taught': [{'course_id': 'LING419W', 'semester': '201908'}]}, {'name': 'Aaron Finkle', 'taught': [{'course_id': 'ECON306', 'semester': '201901'}, {'course_id': 'ECON456', 'semester': '201901'}, {'course_id': 'ECON488K', 'semester': '201901'}, {'course_id': 'ECON641', 'semester': '201901'}, {'course_id': 'BUSO604', 'semester': '201905'}, {'course_id': 'ECON306', 'semester': '201908'}, {'course_id': 'ECON456', 'semester': '201908'}, {'course_id': 'ECON488K', 'semester': '201908'}, {'course_id': 'ECON641', 'semester': '201908'}, {'course_id': 'ECON306', 'semester': '202001'}, {'course_id': 'ECON456', 'semester': '202001'}, {'course_id': 'ECON488K', 'semester': '202001'}, {'course_id': 'ECON641', 'semester': '202001'}, {'course_id': 'BUSO604', 'semester': '202005'}, {'course_id': 'ECON306', 'sem